In [1]:
import numpy as np
import pandas as pd

import random
import math
from decimal import Decimal
import copy

In [2]:
def generate_location():
    loc = []
    loc.append(random.random())
    loc.append(random.random())
    loc.append(random.random()*0.2)
    return(loc)

def generate_rotation():
    rot = []
    rot.append(math.pi*random.random())
    rot.append((math.pi/2)*random.random())
    rot.append(math.pi*random.random())
    return(rot)

In [3]:
# передается размер кубика в формате float
# возвращается список количества ячеек по каждой оси
def cells_number(cube_size_float):
    cube_size = Decimal(str(cube_size_float))
    if 1%(2*cube_size) == 0:
        n_xy = 1//(2*cube_size)
    else:
        n_xy = 1//(2*cube_size)+1
    if Decimal("0.2")%(2*cube_size) == 0:
        n_z = Decimal('0.2')//(2*cube_size)
    else:
        n_z = Decimal('0.2')//(2*cube_size)+1
    number = [int(n_xy), int(n_z)]
    return(number)

In [4]:
def cell_number(loc_float, cube_size_float, n_xy):
    cube_size = Decimal(str(cube_size_float))
    loc = []
    for location in loc_float:
        loc.append(Decimal(str(location)))
    cell_num = loc[0]//(2*cube_size) + (loc[1]//(2*cube_size))*n_xy + (loc[2]//(2*cube_size))*n_xy**2
    cell_num = int(cell_num)
    return(cell_num)

In [5]:
def near_cells(cell, n_xy, n_z):
        x = []
        
        x.append(cell)
        x.append(cell+1)
        x.append(cell-1)
        
        x.append(cell+n_xy)
        x.append(cell-n_xy)
        x.append(cell+1+n_xy)
        x.append(cell-1+n_xy)
        x.append(cell+1-n_xy)
        x.append(cell-1-n_xy)
        
        x.append(cell+n_xy**2)
        x.append(cell+1+n_xy**2)
        x.append(cell-1+n_xy**2)
        
        x.append(cell+n_xy+n_xy**2)
        x.append(cell-n_xy+n_xy**2)
        x.append(cell+1+n_xy+n_xy**2)
        x.append(cell-1+n_xy+n_xy**2)
        x.append(cell+1-n_xy+n_xy**2)
        x.append(cell-1-n_xy+n_xy**2)
        
        x.append(cell-n_xy**2)
        x.append(cell+1-n_xy**2)
        x.append(cell-1-n_xy**2)
        
        x.append(cell+n_xy-n_xy**2)
        x.append(cell-n_xy-n_xy**2)
        x.append(cell+1+n_xy-n_xy**2)
        x.append(cell-1+n_xy-n_xy**2)
        x.append(cell+1-n_xy-n_xy**2)
        x.append(cell-1-n_xy-n_xy**2)
        a = copy.copy(x)
        for i in a:
            if (i<0) or (i>=(n_xy**2*n_z)):
                x.remove(i)
        return(x)

In [6]:
def vertices_loc(loc, size):
    coord_list = []
    
    coord_list.append([loc[0] + size / 2, loc[1] + size / 2, loc[2] + size / 2])
    coord_list.append([loc[0] + size / 2, loc[1] + size / 2, loc[2] - size / 2])
    coord_list.append([loc[0] + size / 2, loc[1] - size / 2, loc[2] + size / 2])
    coord_list.append([loc[0] + size / 2, loc[1] - size / 2, loc[2] - size / 2])
    coord_list.append([loc[0] - size / 2, loc[1] + size / 2, loc[2] + size / 2])
    coord_list.append([loc[0] - size / 2, loc[1] + size / 2, loc[2] - size / 2])
    coord_list.append([loc[0] - size / 2, loc[1] - size / 2, loc[2] + size / 2])
    coord_list.append([loc[0] - size / 2, loc[1] - size / 2, loc[2] - size / 2])
            
    return coord_list

In [7]:
def turn(coord, rot):
    a = rot[0]*(math.pi/180)
    b = rot[1]*(math.pi/180)
    c = rot[2]*(math.pi/180)
    coord_list = []
    M = np.array([ [np.cos(a)*np.cos(c)-np.cos(b)*np.sin(a)*np.sin(c), -np.cos(c)*np.sin(a)-np.cos(a)*np.cos(b)*np.sin(c), np.sin(b)*np.sin(c)],
                  [np.cos(b)*np.cos(c)*np.sin(a)+np.cos(a)*np.sin(c), np.cos(a)*np.cos(b)*np.cos(c)-np.sin(a)*np.sin(c), -np.cos(c)*np.sin(b)],
                  [np.sin(a)*np.sin(b), np.cos(a)*np.sin(b), np.cos(b)] ])
    for c in coord:
        vert = np.array(c)
        new_vert = vert.dot(M)
        coord_list.append(new_vert.tolist())
    return coord_list

In [8]:
def turn_back(coord, rot):
    a = rot[0]*(math.pi/180)
    b = rot[1]*(math.pi/180)
    c = rot[2]*(math.pi/180)
    coord_list = []
    M = np.array([ [np.cos(a)*np.cos(c)-np.cos(b)*np.sin(a)*np.sin(c), -np.cos(c)*np.sin(a)-np.cos(a)*np.cos(b)*np.sin(c), np.sin(b)*np.sin(c)],
                  [np.cos(b)*np.cos(c)*np.sin(a)+np.cos(a)*np.sin(c), np.cos(a)*np.cos(b)*np.cos(c)-np.sin(a)*np.sin(c), -np.cos(c)*np.sin(b)],
                  [np.sin(a)*np.sin(b), np.cos(a)*np.sin(b), np.cos(b)] ])
    for c in coord:
        vert = np.array(c)
        new_vert = vert.dot(M.T)
        coord_list.append(new_vert.tolist())
    return coord_list

In [9]:
def if_intersept(loc1, loc2, rot1, rot2, cube_size):
    
    if ((loc1[0]-loc2[0])**2 + (loc1[1]-loc2[1])**2 + (loc1[2]-loc2[2])**2)**(1/2) > (cube_size*(3**(1/2))):
        return False
    
    else:
        coord_list1 = vertices_loc(loc1, cube_size)
        coord_list2 = vertices_loc(loc2, cube_size)
        
        coord_list2 = turn(coord_list2, rot2)
        coord_list2 = turn_back(coord_list2, rot1)
        
        x1_list = []
        y1_list = []
        z1_list = []

        for c in coord_list1:
            x1_list.append(c[0])
            y1_list.append(c[1])
            z1_list.append(c[2])

        x2_list = []
        y2_list = []
        z2_list = []

        for c in coord_list2:
            x2_list.append(c[0])
            y2_list.append(c[1])
            z2_list.append(c[2])
        
        if ((max(x1_list) >= min(x2_list)) and (min(x1_list) <= max(x2_list))
                and (max(y1_list) >= min(y2_list)) and (min(y1_list) <= max(y2_list))
                and (max(z1_list) >= min(z2_list)) and (min(z1_list)) <= max(z2_list)):
            return False
        else:
            return True

In [11]:
grain_sizes = [0.05]

file_number = 0

for cubeRadius in grain_sizes:
    file_number += 1
    
    n_xyz = cells_number(cubeRadius)
    n_xy = n_xyz[0]
    n_z = n_xyz[1]
    div_number = (n_xy ** 2) * n_z
    
    cubes = []
    rotations = []
    for i in range(div_number):
        cubes.append([])
        rotations.append([])
        
    loc = generate_location()
    rot = generate_rotation()
    
    cubes[cell_number(loc, cubeRadius, n_xy)].append(loc)
    rotations[cell_number(loc, cubeRadius, n_xy)].append(rot)
    
    size = (0.2 / cubeRadius ** 3) * (3 / 7)
    c = 0
    while c <= size:
        while True:
            flag = True
            loc = generate_location()
            rot = generate_rotation()
            cell = cell_number(loc, cubeRadius, n_xy)
            near_grids = near_cells(cell, n_xy, n_z)
            
            for i in near_grids:
                if len(cubes[i]):
                    for l in range(len(cubes[i])):
                        if if_intersept(loc, cubes[i][l], rot, rotations[i][l], cubeRadius) is True:
                            flag = False
            if flag is True:
                break
        
        cubes[cell].append(loc)
        rotations[cell].append(rot)
        c+=1
        
    with open("list_" + str(file_number) + ".txt", "w") as file:
        file.write(str(cubes)+"\n")
        file.write(str(rotations))